<a href="https://colab.research.google.com/github/abhikashyap/projects/blob/main/value_today_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting html file from github

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
response=requests.get("https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page=0")

In [4]:
len(response.text)

170955

In [5]:
page_content=response.text

## Converting to html with beautifulsoup

In [6]:
soup=BeautifulSoup(page_content,"html.parser")

# extracting data


In [183]:
soups=[]
i=0
response=requests.get(f"https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page={i}")
page_content=response.text
soup=BeautifulSoup(page_content,"html.parser")
while i != -1:
  if soup.find_all("li", {"class": "pager__item pager__item--next"}):
    response=requests.get(f"https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page={i}")
    page_content=response.text
    soup=BeautifulSoup(page_content,"html.parser")
    soups.append(soup)

    print(f"https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page={i}")
    i+=1
  else:
    i = -1

https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page=0
https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page=1
https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page=2
https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&page=3
https://www.value.today/headquarters/india?title=&field_headquarters_of_company_target_id&field_company_category_primary_target_id&field_company_website_uri=&field_market_value_jan072022_value=&pa

In [219]:
companies=[]
for soup in soups: 
  company=soup.find_all("li","row well clearfix")
  for element in company:
    companies.append(element)

In [220]:
len(companies)

1000

In [186]:
def find_tag(file,tag,class_= None):
  if file.find(tag,class_):
    return file.find(tag,class_)
  else:
    return "null"

In [210]:
data=companies[1].find("div","clearfix col-sm-6 field field--name-field-market-value-jan072022 field--type-float field--label-above")

In [211]:
if data == None:
  ans = "null"
else:
  file = data
  ans=file.find("div","field--item").text

In [212]:
ans

'220.760 Billion USD'

In [215]:
len(companies)

10

In [224]:
out=[]
for company in companies:
  data=company.find("div","clearfix col-sm-6 field field--name-field-market-value-jan072022 field--type-float field--label-above")
  if data == None:
    ans = "null"
    out.append(ans)
  else:
    file = data
    ans=file.find("div","field--item").text
    out.append(ans)

In [246]:
# extracts information
def extract(class_):
  out=[]
  for company in companies:
    data=company.find("div",class_)
    if data == None:
      ans = "null"
      out.append(ans)
    else:
      file = data
      ans=file.find("div","field--item").text
      out.append(ans)
  return(out)

In [254]:
# extract company_name
company_name=[]
for company in companies:
  company_name.append(company.find("div","field field--name-node-title field--type-ds field--label-hidden field--item").text.strip())

In [258]:
data_frame=[]

In [259]:
data_frame.append(company_name)

In [260]:
# extract World Rank (Jan-07-2022)
world_rank_2022=extract("clearfix col-sm-6 field field--name-field-world-rank-jan072022 field--type-integer field--label-above")

In [262]:
data_frame.append(world_rank_2022)

In [267]:
# extract market value
market_value_2022 = extract("clearfix col-sm-6 field field--name-field-market-value-jan072022 field--type-float field--label-above")

In [268]:
data_frame.append(market_value_2022)

In [ ]:
# extract